In [ ]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
import datetime
from time import sleep

In [ ]:
KEY = ''

RESPONSE_504 = '<Response [504]>'

df = pd.DataFrame(columns=('titles', 'publicationDates', 'abstracts', 'keywords'))
batch_counter = range(427000, 742450, 100)

In [ ]:
for ii in batch_counter:
    
    while True:
        response = requests.get('http://api.springernature.com/meta/v2/json', params={
            's': ii,
            'p': ii+99,
            'q': 'year:2020',
            'api_key': KEY})
        
        if str(response) == RESPONSE_504:
            print('504 error, wait 30 seconds')
            sleep(30)
        else:
            break
        
    js_dict = json.loads(response.text)

    keywords = []
    for i in js_dict['records']:
        try:
            att = i['keyword']
        except KeyError:
            keywords.append(['<No keywords>'])
        else:
            if isinstance(i['keyword'],list):
                keywords.append(i['keyword'])
            else:
                keywords.append([i['keyword']])
                
    titles = [i['title'] for i in js_dict['records']]
    publicationDates = [i['publicationDate'] for i in js_dict['records']]
    abstracts = [i['abstract'] for i in js_dict['records']]
    
    temp_df = pd.DataFrame({'titles':titles, 'publicationDates':publicationDates, \
                            'abstracts':abstracts, 'keywords':keywords})
    temp_df.index = range(ii, ii+100)
    
    df = pd.concat((df, temp_df), axis=0)
    
    print('step =', str(ii), 'time =', datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S"))

print('Mining completed!!!')

In [ ]:
df.to_parquet('mining.parquet')